In [43]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import warnings

In [44]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [45]:
df = pd.read_csv('Dataset.csv')
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [46]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [47]:
rating_dict = {'Not rated': 0, 'Poor': 1, 'Average': 2, 'Good': 3, 'Very Good': 4, 'Excellent': 5}
df['Rating text'] = df['Rating text'].map(rating_dict)
rating_dict_02 = {'White': 0, 'Red': 1, 'Orange': 2, 'Yellow': 3, 'Green': 4, 'Dark Green': 5}
df['Rating color'] = df['Rating color'].map(rating_dict_02)
yes_no = {'Yes': 1, 'No': 0}
df['Has Online delivery'] = df['Has Online delivery'].map(yes_no)
df['Has Table booking'] = df['Has Table booking'].map(yes_no)
df['Is delivering now'] = df['Is delivering now'].map(yes_no)
df['Switch to order menu'] = df['Switch to order menu'].map(yes_no)
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),1,0,0,0,3,4.8,5,5,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),1,0,0,0,3,4.5,5,5,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),1,0,0,0,4,4.4,4,4,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),0,0,0,0,4,4.9,5,5,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),1,0,0,0,4,4.8,5,5,229


In [48]:
country = df

In [50]:
cords = country[['Aggregate rating', 'Price range']]
cords

,Aggregate rating,Price range
0,4.8,3
1,4.5,3
2,4.4,4
3,4.9,4
4,4.8,4
...,...,...
9546,4.1,3
9547,4.2,3
9548,3.7,4
9549,4.0,4


In [51]:
df['Cuisines'].unique

<bound method Series.unique of 0             French, Japanese, Desserts
1                               Japanese
2       Seafood, Asian, Filipino, Indian
3                        Japanese, Sushi
4                       Japanese, Korean
                      ...               
9546                             Turkish
9547     World Cuisine, Patisserie, Cafe
9548              Italian, World Cuisine
9549                     Restaurant Cafe
9550                                Cafe
Name: Cuisines, Length: 9551, dtype: object>

In [52]:
sil = []
kmax = 50
for k in range(2,kmax+1):
    kmeans = KMeans(n_clusters = k).fit(cords)
    labels = kmeans.labels_
    sil.append(silhouette_score(cords,labels,metric='euclidean'))

sil

len(sil)

49

In [37]:
kmeans = KMeans(n_clusters=len(sil),init='k-means++')
kmeans.fit(cords)
y = kmeans.labels_

In [53]:
country['cluster']=kmeans.predict(country[['Aggregate rating','Price range']])

In [54]:
country.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes,cluster
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,1,0,0,0,3,4.8,5,5,314,29
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,1,0,0,0,3,4.5,5,5,591,11
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,1,0,0,0,4,4.4,4,4,270,49
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,0,0,0,0,4,4.9,5,5,365,4
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,1,0,0,0,4,4.8,5,5,229,4


In [55]:
length = len(country)

def recommend_restaurants(df,Aggregate_rating,Price_range,count=10):
    cluster = kmeans.predict(np.array([Aggregate_rating,Price_range]).reshape(1,-1))[0]
    if length<count:
       count = len(top_restuarants)
        
    df=df[df['cluster']==cluster].iloc[0:count][['Restaurant Name','Aggregate rating','Average Cost for two','Price range','City',"Cuisines"]]
    top_restuarants = df.sort_values(by=["Price range"],ascending=False)

    return(top_restuarants)

In [41]:
top_3_places = [
    [4.2,3], 
    [3.8,3], 
    [4.6,2]
    ]

In [56]:
result = recommend_restaurants(country,top_3_places[1][0],top_3_places[1][1],count=10)

/home/shivek/.local/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [57]:
result

,Restaurant Name,Aggregate rating,Average Cost for two,Price range,City,Cuisines
25,Maori,3.8,60,3,Bras�_lia,Brazilian
28,Beirute,3.7,90,3,Bras�_lia,Arabian
68,Red Steak & Burger,3.9,75,3,S��o Paulo,"Brazilian, Grill"
99,The Catch Seafood Room & Oyster Bar,3.8,40,3,Albany,"Seafood, Tapas, Bar Food"
128,Manuel's Bread Cafe,3.8,40,3,Augusta,"Breakfast, French"
165,Winifreds,3.7,40,3,Cedar Rapids/Iowa City,"American, Mediterranean, Seafood"
178,Graze,3.8,40,3,Cedar Rapids/Iowa City,"American, International, Sushi"
193,Fuji Japanese Steak House,3.9,40,3,Columbus,"Japanese, Steak"
196,Wasabi Sushi and Thai,3.7,40,3,Columbus,"Japanese, Sushi, Thai"
215,Fuji Japanese Steakhouse,3.8,40,3,Dalton,"Japanese, Sushi"
